In [ ]:
import pandas as pd
import json
from nltk import ngrams
from joblib import load

In [5]:
from genson import SchemaBuilder
import json

with open('/workspace/data/source/example-api-json.json', 'r') as fl:
    json_example = json.load(fl)

builder = SchemaBuilder()
builder.add_object(json_example)
builder.to_json()


'{"$schema": "http://json-schema.org/schema#", "type": "object", "properties": {"classifers": {"type": "array", "items": {"type": "object", "properties": {"id": {"type": "integer"}, "name": {"type": "string"}, "labels": {"type": "array", "items": {"type": "string"}}}, "required": ["id", "labels", "name"]}}, "results": {"type": "array", "items": {"type": "object", "properties": {"recordIdentifier": {"type": "string"}, "sentence": {"type": "string"}, "classifiers": {"type": "array", "items": {"type": "object", "properties": {"id": {"type": "integer"}, "confidence": {"type": "array", "items": {"type": "number"}}}, "required": ["confidence", "id"]}}}, "required": ["classifiers", "recordIdentifier", "sentence"]}}}, "required": ["classifers", "results"]}'

# Generating monitoring data

## Context

The aim of this demo is to show how Scenario evaluation data, collected over time, can be used to create a model monitoring dashboard. The dashboard should show the following as a minimum view inot model behaviour:

1. *Predicted confidence score distrobutions over time*  
Plan is to simulate a scenario where there is a drastic drop in the frequency of secrecy 
2. *ngram histogram over time*  
This might have to be restricted to positive ngrams only for visual purposes

As a stretch, it might be valuable to show how confidence score distrobutions change in response to a pipeline change (e.g. incorrect ngram range causes bi-grams to be dropped from the inference inputs).

## Methodology

To simulate a relevant business process for monitoring material behviour of the scenario model, we will leverage the secrecy test dataset. We will control the confidence score distribution by over and under-sampling the positive language of interest in the test set. When we over-sample positives, we expect the confidence score distribution to be skewed toward higher scores, while under-sampling will have the opposite effect.

### Temporal monitoring

In the business context we will use for the demo, the hypothetical client has built a dashboard that ingests Scenario evaluations, and visualises confidence score distrobutions, along with ngram histograms. Scenario evaluations are run manually, but can be automatically ingested by the dashboard app. We will visualise the distrobutions quarterly to give a termoral angle to the monitoring.

In [ ]:
# load the secrecy model for inference
model = load(f"../models/{MODEL_NAME}_pipeline.joblib")